### **slope_models**

Run hierarchical models on project data:
- Test hypothesis (a), that in association cortices, the aperiodic slope flattens with age into young adulthood.
- Test hypothesis (b), that in sensorimotor cortices, the aperiodic slope flattens with age into adolescence.
- Test hypothesis (c), that regionally specific age effects differ between attentional states.

Author: EL Johnson, PhD

Copyright (c) 2022-2025  
ZR Cross, PhD & EL Johnson, PhD

### Load packages:

In [ ]:
library(tidyverse)
library(RColorBrewer)
library(broom)
library(lme4)
library(lmerTest)
library(splines)
library(ggridges)

### Set directories:

In [ ]:
pth <- getwd()  # navigate to directory containing folders of downloaded data
datdir <- paste(pth, 'Preprocessed_data', sep = '/')
savdir <- datdir  # replace with your save path

### Load data:

In [ ]:
dat_load <- paste(datdir, 'aperiodic_slope_models.csv', sep = '/')
average_df <- read.csv(dat_load) %>% 
  select(-X)

### Plot distribution of slopes:

In [ ]:
# convert DKT to factor with reverse alphabetical order
average_df$DKT <- factor(average_df$DKT, levels = rev(unique(average_df$DKT[order(average_df$DKT)])))

# plot with reverse alphabetically ordered DKT
ggplot(average_df, aes(x = Slope, y = DKT, fill = after_stat(x))) +
  geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01) +
  labs(y = '', x = 'Aperiodic slope') +
  facet_grid(~condition) +
  theme_minimal() +
  theme(
    panel.spacing = unit(0.1, 'lines'),
    strip.text.x = element_text(size = 14, colour = 'black'),
    strip.text.y = element_text(size = 14, colour = 'black'),
    legend.position = 'none',
    legend.spacing.x = unit(0.2, 'cm'),
    legend.key.size = unit(0.4, 'cm'),
    legend.background = element_rect(fill = alpha('blue', 0)),
    legend.text = element_text(size = 10, colour = 'black'),
    legend.title = element_text(size = 10, colour = 'black', face = 'bold'),
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14),
    plot.title = element_text(size = 12, face = 'bold'))

### Get demographic data:

In [ ]:
# get number of males/females by condition
sex_condition <- average_df %>%
  ungroup() %>% 
  select(subj, sex, age) %>%
  group_by(sex, subj) %>% 
  dplyr::summarise(age = mean(age)) %>% 
  dplyr::count(sex, sort = TRUE)

# make unique subjects into a df
sid_list <- unique(average_df$subj)
sid_list <- as.data.frame(sid_list)

# check length of channels
length(unique(average_df$realID))

# get information for age
summary(average_df$age)

# remove subcortical b/c there aren't enough observations to model
average_df <- average_df %>% 
  filter(DKT != 'subcortical')

### Run model to test hypotheses (a) and (b):

In [ ]:
large_df <- average_df %>%
      filter(DKT != 'amygdala' & DKT != 'hippocampus') %>%
      select(Slope, Offset, subj, DKT, age, realID) %>%
      mutate(Cortex = case_when(
          DKT %in% c('caudal middle frontal', 'insula', 'caudal anterior cingulate',
               'fusiform', 'inferior temporal', 'inferior parietal', 
               'superior temporal', 'middle temporal', 'parahippocampal', 
               'rostral middle frontal', 'posterior cingulate', 
               'superior parietal', 'medial orbitofrontal', 
               'inferior frontal', 'lateral orbitofrontal', 'temporal pole') ~ 'Association',
          DKT %in% c('postcentral', 'precentral', 'lateral occipital') ~ 'Sensorimotor',
          TRUE ~ NA_character_   # in case there are other regions not covered
      )) %>%
      group_by(subj, realID, Cortex, DKT) %>%
      dplyr::summarise(Slope = mean(Slope),
                       Offset = mean(Offset),
                       age = mean(age)) %>% 
      na.omit()

large_df$subj <- as.factor(large_df$subj)
large_df$realID <- as.factor(large_df$realID)
large_df$Cortex <- as.factor(large_df$Cortex)
large_df$DKT <- as.factor(large_df$DKT)

large_model_slope <- lmer(Slope ~ ns(age, df = 2) * Cortex + (1|subj) + (1|DKT), data = large_df, control = lmerControl(optimizer = 'bobyqa'))
summary(large_model_slope)

### Save large model:

In [ ]:
dat_save <- paste(savdir, 'large_model_slope.RData', sep = '/')
save(large_model_slope, file = dat_save)

### Run model to test hypothesis (c):

In [ ]:
# loop through DKT regions
run_lmm_and_extract_effects <- function(data) {
    data <- na.omit(data)   # filter out rows with missing values
    if (nrow(data) == 0) {
        cat('Skipping: no non-NA data available for this DKT region.\n')
        return(data.frame())   # return an empty df if no data
    }
    
    model <- lmer(Slope ~ age * condition + (1 | subj/realID) + (1|task), data = data, control = lmerControl(optimizer = 'bobyqa'))
    
    # extract fixed effects summary
    return(as.data.frame(summary(model)$coefficients))
}

### Create an empty list to populate with DKT labels:

In [ ]:
DKT_levels <- unique(average_df$DKT)
combined_results <- data.frame()  # create an empty df for combined results
results_df_slope <- data.frame()  # create an empty df for results

### Apply lmer function to each DKT region:

In [ ]:
for (level in DKT_levels) {
    cat('Processing DKT level:', level, '\n')
    
    # subset the data
    subset_data <- average_df[average_df$DKT == level, ]
    
    # print the number of rows and a sample of the subset data for debugging
    cat('Number of rows in subset_data:', nrow(subset_data), '\n')
    print(head(subset_data))
    
    # check if there's enough data to run the model
    if (nrow(subset_data) < 5) {  # threshold can be adjusted as needed
        cat('Skipping:', level, '- Not enough data.\n')
        next  # skip this iteration if not enough data
    }
    
    # run lmer model and extract effects
    results <- run_lmm_and_extract_effects(subset_data)
    if (nrow(results) == 0) {
        cat('Skipping:', level, '- No results returned.\n')
        next  # skip if no results returned
    }
    
    # add the DKT level as a column in the results
    results$DKT <- level
    # append results to combined_results
    combined_results <- rbind(combined_results, results)
    
    # run lmer model for slope results
    model <- lmer(Slope ~ age * condition + (1 | subj/realID) + (1|task), data = subset_data, control = lmerControl(optimizer = 'bobyqa'))
    summary_model <- summary(model)
    
    # extract estimates, standard errors, and p-values for the fixed effects
    fixed_effects <- summary_model$coefficients[c(2, 3, 4), ]
    
    # create a new df to store the results for slope
    result_row <- data.frame(
        DKT = level, 
        Estimate_Age = fixed_effects['age', 'Estimate'], 
        SE_Age = fixed_effects['age', 'Std. Error'], 
        PValue_Age = fixed_effects['age', 'Pr(>|t|)'], 
        Estimate_Condition = fixed_effects['conditionTask', 'Estimate'], 
        SE_Condition = fixed_effects['conditionTask', 'Std. Error'], 
        PValue_Condition = fixed_effects['conditionTask', 'Pr(>|t|)'], 
        Estimate_Interaction = fixed_effects['age:conditionTask', 'Estimate'], 
        SE_Interaction = fixed_effects['age:conditionTask', 'Std. Error'], 
        PValue_Interaction = fixed_effects['age:conditionTask', 'Pr(>|t|)']
        )
    
    # append the results to the main df for slope
    results_df_slope <- rbind(results_df_slope, result_row)
}

### Generate table of results:

In [ ]:
slope_table <- results_df_slope %>% 
    dplyr::rename(ROI = DKT, P_Age = PValue_Age, P_Condition = PValue_Condition, P_Interaction = PValue_Interaction)

### Apply FDR correction:

In [ ]:
results_df_slope$p_age_corrected <- p.adjust(results_df_slope$PValue_Age, method = 'fdr')
results_df_slope$p_condition_corrected <- p.adjust(results_df_slope$PValue_Condition, method = 'fdr')
results_df_slope$p_interaction_corrected <- p.adjust(results_df_slope$PValue_Interaction, method = 'fdr')

### Save model results:

In [ ]:
dat_save <- paste(savdir, 'results_df_slope.csv', sep = '/')
write.csv(results_df_slope, dat_save, row.names = FALSE)

### Plot age effects:

In [ ]:
# age df for plotting
age_df <- average_df %>% 
    select(subj, DKT, Slope, age) %>% 
    group_by(subj, DKT) %>% 
    summarise(Slope = mean(Slope), age = mean(age))

# plot all DKT regions
ggplot(age_df, aes(x = age, y = Slope)) + 
      geom_point(color = 'brown3', alpha = 0.6, size = 2) +  # change to brick red
      geom_smooth(method = 'lm', linetype = 'solid', color = 'darkgray') +
      geom_ribbon(aes(ymin = after_stat(ymin), ymax = after_stat(ymax)), stat = 'smooth', method = 'lm', fill = 'darkgray', alpha = 0.1) +
      scale_color_viridis_c(option = 'magma') +
      ylab('Aperiodic slope') + xlab ('Age [years]') +
      theme_classic() +
      facet_wrap(~DKT) +
      theme(legend.position = 'none',
            legend.spacing.x = unit(0.1, 'cm'),
            legend.key.size = unit(0.5, 'cm'),
            legend.background = element_rect(fill = alpha('blue', 0)),
            legend.text = element_text(size = 8, colour = 'black'),
            legend.title = element_text(size = 8, colour = 'black'),
            strip.text.x = element_text(size = 8, colour = 'black'),
            strip.text.y = element_text(size = 8, colour = 'black'),
            axis.text = element_text(size = 8),
            axis.title = element_text(size = 12),
            plot.title = element_text(size = 8, face = 'bold'))

### Plot age*attentional state effects:

In [ ]:
# age df for plotting - keep condition separate
age_df <- average_df %>%
    select(subj, DKT, Slope, age, condition) %>% 
    group_by(subj, DKT, condition) %>% 
    summarise(Slope = mean(Slope), age = mean(age), .groups = 'drop')

# plot all DKT regions with conditions
ggplot(age_df, aes(x = age, y = Slope, color = condition)) + 
    geom_point(alpha = 0.6, size = 2) +
    geom_smooth(method = 'lm', linetype = 'solid') +
    geom_ribbon(aes(ymin = after_stat(ymin), ymax = after_stat(ymax), fill = condition), stat = 'smooth', method = 'lm', alpha = 0.1, color = NA) +
    scale_color_manual(values = c('Task' = 'brown3', 'Rest' = 'darkgray')) +
    scale_fill_manual(values = c('Task' = 'brown3', 'Rest' = 'darkgray')) +
    ylab('Aperiodic slope') + xlab ('Age [years]') +
    theme_classic() +
        facet_wrap(~DKT) +
        theme(legend.position = 'bottom',
        legend.spacing.x = unit(0.1, 'cm'),
        legend.key.size = unit(0.5, 'cm'),
        legend.background = element_rect(fill = alpha('blue', 0)),
        legend.text = element_text(size = 8, colour = 'black'),
        legend.title = element_text(size = 8, colour = 'black'),
        strip.text.x = element_text(size = 8, colour = 'black'),
        strip.text.y = element_text(size = 8, colour = 'black'),
        axis.text = element_text(size = 8),
        axis.title = element_text(size = 12),
        plot.title = element_text(size = 8, face = 'bold'))